# Species Prior
PaCMAP repo: https://github.com/YingfanWang/PaCMAP

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from plantclef.spark import get_spark

spark = get_spark()
display(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/14 00:35:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/14 00:35:56 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
25/04/14 00:35:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### embeddings

In [3]:
import os
from pathlib import Path

# Get list of stored filed in cloud bucket
root = Path(os.path.expanduser("~"))
! date

Mon Apr 14 12:35:59 AM EDT 2025


### test embeddings

In [4]:
# Path and dataset names
data_path = f"{root}/p-dsgt_clef2025-0/shared/plantclef/data/embeddings"

# Define the path to the train and test parquet files
test_path = f"{data_path}/test_2025/test_2025_embed_logits"

# Read the parquet files into a spark DataFrame
test_df = spark.read.parquet(test_path)

# Show the data
test_df.printSchema()
test_df.show(n=5)

root
 |-- image_name: string (nullable = true)
 |-- output: struct (nullable = true)
 |    |-- cls_token: array (nullable = true)
 |    |    |-- element: float (containsNull = true)
 |    |-- logits: array (nullable = true)
 |    |    |-- element: float (containsNull = true)
 |-- sample_id: integer (nullable = true)

+--------------------+--------------------+---------+
|          image_name|              output|sample_id|
+--------------------+--------------------+---------+
|CBN-Pla-A1-201908...|{[0.47354543, 1.5...|        0|
|CBN-Pla-D6-201908...|{[-0.39621377, 1....|        0|
|CBN-PdlC-C5-20140...|{[-0.5331654, 0.2...|        0|
|LISAH-BOU-0-37-20...|{[1.2480925, 0.47...|        0|
|CBN-Pla-E4-201308...|{[0.7065191, 1.70...|        0|
+--------------------+--------------------+---------+
only showing top 5 rows



In [7]:
import pickle
import numpy as np


# model directory
model_dir = "~/p-dsgt_clef2025-0/shared/plantclef/models/pacmap"

# load the pacmap model
pipeline_filename = Path(
    os.path.expanduser(f"{model_dir}/plant_clustering_pipeline.pkl")
)
with open(pipeline_filename, "rb") as file:
    loaded_pipeline = pickle.load(file)

# Convert to Pandas DF
col_name = "output.cls_token"
df = test_df.select([col_name, "grouped_regions"])
pandas_df = df.select(["cls_token", "grouped_regions"]).toPandas()

# extract embeddings
cls_token = np.stack(pandas_df["cls_token"].values)

# Use the loaded pipeline to predict on new data
cluster_labels = loaded_pipeline.predict(cls_token)
cluster_labels[:10]

AttributeError: Can't get attribute 'PaCMAPTransformer' on <module '__main__'>